# Working with individual PASTIS modes and their influence on the DZ

In [ ]:
# Imports
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline
from astropy.io import fits
import astropy.units as u
import hcipy as hc
from hcipy.optics.segmented_mirror import SegmentedMirror

os.chdir('../../pastis/')
import util_pastis as util
from e2e_simulators.luvoir_imaging import LuvoirAPLC

In [ ]:
eunit = 1e-9

## Instantiate Segmented mirror for plotting of primary

In [ ]:
# Load aperture files needed for SM
nseg = 120
wvln = 638e-9

datadir = '/Users/ilaginja/Documents/LabWork/ultra/LUVOIR_delivery_May2019/'
aper_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000.fits'
aper_ind_path = 'inputs/TelAp_LUVOIR_gap_pad01_bw_ovsamp04_N1000_indexed.fits'
aper_read = hc.read_fits(os.path.join(datadir, aper_path))
aper_ind_read = hc.read_fits(os.path.join(datadir, aper_ind_path))

pupil_grid = hc.make_pupil_grid(dims=aper_ind_read.shape[0], diameter=15)
aper = hc.Field(aper_read.ravel(), pupil_grid)
aper_ind = hc.Field(aper_ind_read.ravel(), pupil_grid)

wf_aper = hc.Wavefront(aper, wvln)

# Load segment positions from fits header
hdr = fits.getheader(os.path.join(datadir, aper_ind_path))

poslist = []
for i in range(nseg):
    segname = 'SEG' + str(i+1)
    xin = hdr[segname + '_X']
    yin = hdr[segname + '_Y']
    poslist.append((xin, yin))
    
poslist = np.transpose(np.array(poslist))
seg_pos = hc.CartesianGrid(poslist)

In [ ]:
# Instantiate SM
sm = SegmentedMirror(aper_ind, seg_pos)

## Instantiate LUVOIR telescope for full functionality

In [ ]:
# Instantiate LUVOIR
sampling = 4
apodizer_design = 'medium'
# This path is specific to the paths used in the LuvoirAPLC class
optics_input = '/Users/ilaginja/Documents/LabWork/ultra/LUVOIR_delivery_May2019/'

luvoir = LuvoirAPLC(optics_input, apodizer_design, sampling)

In [ ]:
# Make reference image
luvoir.flatten()
psf_unaber, ref = luvoir.calc_psf(ref=True)
norm = ref.max()

In [ ]:
# Make dark hole
dh_outer = hc.circular_aperture(2*luvoir.apod_dict[apodizer_design]['owa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_inner = hc.circular_aperture(2*luvoir.apod_dict[apodizer_design]['iwa'] * luvoir.lam_over_d)(luvoir.focal_det)
dh_mask = (dh_outer - dh_inner).astype('bool')

plt.figure(figsize=(18, 6))
plt.subplot(131)
hc.imshow_field(psf_unaber/norm, norm=LogNorm())
plt.subplot(132)
hc.imshow_field(dh_mask)
plt.subplot(133)
hc.imshow_field(psf_unaber/norm, norm=LogNorm(), mask=dh_mask)

In [ ]:
dh_intensity = psf_unaber/norm * dh_mask
baseline_contrast = np.mean(dh_intensity[np.where(dh_intensity != 0)])
print('contrast:', baseline_contrast)

In [ ]:
# Which directory are we working in?
savedpath = '/Users/ilaginja/Documents/data_from_repos/pastis_data/2019-8-07_002_1nm'    # small apodizer LUVOIR

# Load eigenvalues - lowest first, highest at the end
evals = np.loadtxt(os.path.join(savedpath, 'results', 'singular_values.txt'))
print('evals.shape: {}'.format(evals.shape))

# Load eigenmodes - piston value per segment per mode
emodes = np.loadtxt(os.path.join(savedpath, 'results', 'singular_modes.txt'))
print('emodes.shape: {}'.format(emodes.shape))

In [ ]:
mode = 115   # We start numbering at 0 here, 0-35 (Python nunmbering!)

sm.flatten()
for seg, val in enumerate(emodes[:, mode]):
    #print(val)
    sm.set_segment(seg+1, eunit*val/2, 0, 0)

# Propagate WF and display SM phase
wf_sm = sm(wf_aper)

hc.imshow_field(wf_sm.phase, cmap='RdBu')
plt.colorbar()

In [ ]:
# its singular values:
print(evals[mode])

In [ ]:
lam_p = evals[mode]
c = 1e-10
sigma_p = np.sqrt(c/lam_p)
print(sigma_p)

In [ ]:
opd = emodes[:, mode] * sigma_p
rms = np.sqrt(np.mean(np.square(opd)) - np.square(np.mean(opd)))
print(np.std(opd))

In [ ]:
luvoir.flatten()
for seg, val in enumerate(opd):
    luvoir.set_segment(seg+1, eunit*val/2, 0, 0)

# Get PSF from putting this OPD on the SM
psf = luvoir.calc_psf()

# Calculate the contrast from that PSF
dh_intensity = psf/norm * dh_mask
contrast = np.mean(dh_intensity[np.where(dh_mask != 0)])
print(contrast)

hc.imshow_field(np.log10(psf))

In [ ]:
print(opd)
print(emodes[:, mode])